In [61]:
import requests
import os
import pandas as pd

from time import sleep

In [62]:
# Load .env environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [63]:
api_domain = "https://api.ethplorer.io"
ethplorer_api_key = "EK-cASkJ-bSrvUqh-Am7L7"

# number of tokens that we want to track
num_tokens_to_watch = 1
# number of top holders that we want to track
num_holders_to_watch = 5

# threshold for sending SMS
whale_alert_threshold = 1

In [64]:
# top tokens by trade volume
top_token_request_url = f"{api_domain}/getTopTokens?apiKey={ethplorer_api_key}"
top_tokens_by_trade_volume = requests.get(top_token_request_url).json().get('tokens')[:num_tokens_to_watch]

# get a map of token name -> address
token_name_to_address = {token['name']: token['address'] for token in top_tokens_by_trade_volume}
token_name_to_address

{'WETH': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}

In [65]:
top_holder_addresses = {}

# for each token
for token, address in token_name_to_address.items():
    # find the top holders
    top_holders_request_url = f"{api_domain}/getTopTokenHolders/{address}?apiKey={ethplorer_api_key}&limit={num_holders_to_watch}"
    top_holders = requests.get(top_holders_request_url).json().get('holders')
    if top_holders:
        top_holder_addresses[token] = [holder['address'] for holder in top_holders]
        
top_holder_addresses

{'WETH': ['0x2f0b23f53734252bda2277357e97e1517d6b042a',
  '0x030ba81f1c18d280636f32af80b9aad02cf0854e',
  '0xf04a5cc80b1e94c69b48f5ee68a08cd2f09a7c3e',
  '0x94b0a3d511b6ecdb17ebf877278ab030acb0a878',
  '0xceff51756c56ceffca006cd410b03ffc46dd3a58']}

In [66]:
!pip install twilio

In [67]:
from twilio.rest import Client

In [68]:
# Your Account Sid and Auth Token from twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = os.getenv('TWILIO_ACCOUNT_SID')
auth_token = os.getenv('TWILIO_AUTH_TOKEN')
client = Client(account_sid, auth_token)

In [69]:
timestamp_offset = {}

for token, addresses in top_holder_addresses.items():
    for addr in addresses:
        request_url = f"{api_domain}/getAddressTransactions/{addr}?apiKey={ethplorer_api_key}&limit=1000"
        if addr in timestamp_offset:
            request_url += f"&timestamp={timestamp_offset}"
            
        response = requests.get(request_url).json()
        
        # continue if we run into rate limit errors
        if not isinstance(response, list):
            # print error for debugging purposes
            print(response)
            continue
            
        # update timestamp
        if response:
            timestamp_offset[addr] = max(response, key=lambda txn: txn['timestamp'])
            
        # only look at successful transactions
        transactions = list(filter(lambda txn: txn['success'], response))
        
        # and see if we need to send SMS
        if any(txn['value'] > whale_alert_threshold for txn in transactions):
            
            df = pd.DataFrame(transactions)
            # TODO for Alex/Serra:
            #
            # dataframe is ready for visualizatio, see below for the format
            # generate the png and attach that to the SMS message below
            
            body = f'{token} WHALE ALERT! {addr} had a transaction of more than {whale_alert_threshold} ETH.'
            message = client.messages.create(body=body, from_='+18479614395', to='+14124650602‬')
df

,from,hash,input,success,timestamp,to,value
0,0xff44a4ae22c89370c132a5ff51441335938aebb1,0xdab62f305257e5bb41cfdf12eaf06334ecda4dca77a2...,0x,False,1618693545,0x94b0a3d511b6ecdb17ebf877278ab030acb0a878,0.120
1,0x60ee0c8bb493b79286a54997322904f74fba3736,0x339be3728b0520526cac6a9bfa3588f5bee9c070ef10...,0x,False,1617861575,0x94b0a3d511b6ecdb17ebf877278ab030acb0a878,0.002
2,0x79d34f0d56e4b675fce24e957bc22d5ef4333008,0x93b053df9abafb6074f2acd70fcc683241cddd7dac0c...,0x,False,1617861455,0x94b0a3d511b6ecdb17ebf877278ab030acb0a878,0.001
